In [11]:
from googleapiclient.discovery import build

#api_key = 'API KEY'

api_key = ''


video_id = "dQw4w9WgXcQ"

# empty list for storing reply
replies = []

# creating youtube resource object
youtube = build('youtube', 'v3',
developerKey=api_key)

# retrieve youtube video results
video_response=youtube.commentThreads().list(
  part='snippet',
  maxResults=10,
  order="relevance",
  videoId=video_id
).execute()




    # empty reply list
    # replies = []

  # # Again repeat
  # if 'nextPageToken' in video_response:
  #   video_response = youtube.commentThreads().list(
  #   part = 'snippet',
  #   maxResults=10,
  #   videoId = video_id
  #   ).execute()
  # else:
  #   break

# Enter video id
# video_id = "Enter Video ID"




In [45]:
replies = []
# extracting required info
# from each result object
for item in video_response['items']:

  # Extracting comments
  comment = item['snippet']['topLevelComment']['snippet']['textOriginal']
  replies.append(comment)

# print comment with list of reply

In [46]:
replies

['1 BILLION views for Never Gonna Give You Up!\xa0 Amazing, crazy, wonderful! Rick ♥️',
 "This isn't your first time here\nAnd you know it.",
 'Vienes del Rewind 2022? JKAJSK',
 'Rick Astley haciéndose aún más famoso gracias al Rewind Hispano 2022, sencillamente maravilloso\n\nEdit: Gracias por los likes gente, espero que vuestro 2023 sea próspero y mejor que este 2022!',
 'Memes aside, even after 13 years this song still has value and gives that early 200s and 1980s vibe , back when the world was at its best.',
 'Naah... temardo en verdad 😎🤙\n6025619 personas ya lo han escuchado,\nespero que te haya gustado 😉',
 'I definitely did NOT just get Rick Rolled, shut up, you did\n*cries*',
 'Naah... temardo en verdad 😎🤙\r\n2100011 personas ya lo han escuchado,\r\nespero que te haya gustado 😉',
 'Even when’s it’s 2023 thousands of people still listen to this song everyday,it never gets old❤',
 "Honestly I love this song, so whenever i get rick rolled I actually thank the person for sending it

In [39]:
display(video_response)

{'kind': 'youtube#commentThreadListResponse',
 'etag': 'deXgEqFh1eLXx_zM6ZALyUOCm1Q',
 'nextPageToken': 'QURTSl9pMC1SYVJIa0NJU0llWWJka0hhclA2THVQWUVvZkY5dHVXb1ZhUFNDRF92NmRyRUl3U1dlM1NnZWh3SDNZN2huWWVRWGtRdjR3dlhEV0pJdU5lbVlkckpBNkc3LUw0RGQ3QmRUYklNeFd3WVQ2WnBFekkyVGdEa09VYkJRNDVMVUVBOXpiNERpcmlQVVpUdXlvT29fNFBrOTRIYjFaVUdIbnNWU3Q1Z1NIWVNERm1xWHZaT1hVSVBaWVlCekdILQ==',
 'pageInfo': {'totalResults': 10, 'resultsPerPage': 10},
 'items': [{'kind': 'youtube#commentThread',
   'etag': '4s5OeiB8JMp0ZJMh9bxPv2mlvxY',
   'id': 'UgzarqjaaPC7TbFINNx4AaABAg',
   'snippet': {'videoId': 'dQw4w9WgXcQ',
    'topLevelComment': {'kind': 'youtube#comment',
     'etag': 'dRMIxgYkkkOGHFNWX5wLuaKDBnE',
     'id': 'UgzarqjaaPC7TbFINNx4AaABAg',
     'snippet': {'videoId': 'dQw4w9WgXcQ',
      'textDisplay': '1 BILLION views for Never Gonna Give You Up!\xa0 Amazing, crazy, wonderful! Rick ♥️',
      'textOriginal': '1 BILLION views for Never Gonna Give You Up!\xa0 Amazing, crazy, wonderful! Rick ♥️',
      'a